In [114]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [115]:
ENCODING_DICT = {
            'a': 0,
            'f': 1,
            'e': 2,
            'c': 3,
            'b': 4,
            'h': 5,
            'v': 6,
            'z': 7,
            '2': 8,
            'x': 9,
            'g': 10,
            'm': 11,
            'r': 12,
            'u': 13,
            'p': 14,
            's': 15,
            'd': 16,
            'n': 17,
            '6': 18,
            'k': 19,
            't': 20
            }

DECODING_DICT = {
            0: 'a',
            1: 'f',
            2: 'e',
            3: 'c',
            4: 'b',
            5: 'h',
            6: 'v',
            7: 'z',
            8: '2',
            9: 'x',
            10: 'g',
            11: 'm',
            12: 'r',
            13: 'u',
            14: 'p',
            15: 's',
            16: 'd',
            17: 'n',
            18: '6',
            19: 'k',
            20: 't'
            }

In [116]:
def imagepath_to_captcha(imagepath):
    return imagepath.split("_")[1].split(".")[0]

def to_onehot(captcha):
    label = np.zeros((len(captcha), len(ENCODING_DICT)))
    for index, char in enumerate(captcha):
        label[index, ENCODING_DICT[char]] = 1
    return tf.convert_to_tensor(label, tf.float32)
    # return tf.reshape(tf.convert_to_tensor(label, tf.float32), [-1])

def captcha_from_onehot(tensor):
    label = []
    for index in tf.argmax(tensor, 1).numpy():
        label.append(DECODING_DICT[index])
    return "".join(label)

def load_dataset(filepath):
    filenames = os.listdir(filepath)
    x, y = [], []
    for file in filenames:
        imagepath = filepath + "/" + file
        image = tf.io.read_file(imagepath)
        image = tf.image.decode_image(image)
        pixels = tf.image.convert_image_dtype(image, tf.float32)
        # To show images: 
        # plt.imshow(tf.reshape(pixels, image.shape))
        # plt.title('Image as Pixels')
        # plt.show()
        x.append(pixels)
        captcha = imagepath_to_captcha(file).lower()
        y.append(to_onehot(captcha))
    return (tf.convert_to_tensor(x, tf.float32), tf.convert_to_tensor(y, tf.float32))


# Read input data

In [117]:
train_x, train_y = load_dataset("dataset/train")
test_x, test_y = load_dataset("dataset/test")

print("Training input shape:", train_x.shape)
print("Training output shape:", train_y.shape)
print("Testing input shape:", test_x.shape)
print("Testing output shape:", test_y.shape)

Training input shape: (8501, 50, 250, 3)
Training output shape: (8501, 6, 21)
Testing input shape: (1500, 50, 250, 3)
Testing output shape: (1500, 6, 21)


In [141]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(64, (4, 4), activation='relu', input_shape=(50, 250, 3)))
# model.add(keras.layers.AveragePooling2D(2, 2))
model.add(keras.layers.MaxPooling2D(3, 3))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Conv2D(64, (2, 2), activation='sigmoid'))
# model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
model.add(keras.layers.MaxPooling2D(2, 2))

model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(8064, activation='relu'))
# model.add(keras.layers.Dense(504, activation='relu'))
# model.add(keras.layers.Dense(504, activation=keras.layers.LeakyReLU(alpha=0.1)))
# model.add(keras.layers.Dense(504, activation='relu'))
model.add(keras.layers.Dense(252, activation='relu'))
# model.add(keras.layers.Dense(126, activation='relu'))
model.add(keras.layers.Dense(126, activation='softmax'))
model.add(keras.layers.Reshape((6, 21)))


model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_68 (Conv2D)          (None, 47, 247, 64)       3136      
                                                                 
 max_pooling2d_50 (MaxPooli  (None, 15, 82, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_69 (Conv2D)          (None, 14, 81, 64)        16448     
                                                                 
 max_pooling2d_51 (MaxPooli  (None, 7, 40, 64)         0         
 ng2D)                                                           
                                                                 
 flatten_28 (Flatten)        (None, 17920)             0         
                                                                 
 dense_95 (Dense)            (None, 252)             

In [119]:
# ResNet50
# model = keras.applications.ResNet50(
#     include_top = False,
#     weights = None,
# input_tensor = keras.layers.Input(train_x))

In [142]:
model.compile(
    optimizer = "adam",
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

# graphwiz doesn't install on gLinux ffs.
# tf.keras.utils.plot_model(model, show_shapes=True)


In [143]:
history = model.fit(train_x, train_y,
                    epochs = 5)

print(history.history)

Epoch 1/5
266/266 [==============================] - 14s 52ms/step - loss: 2.5141 - accuracy: 0.2262
Epoch 2/5
266/266 [==============================] - 14s 51ms/step - loss: 0.7975 - accuracy: 0.7542
Epoch 3/5
266/266 [==============================] - 14s 51ms/step - loss: 0.2676 - accuracy: 0.9191
Epoch 4/5
266/266 [==============================] - 14s 51ms/step - loss: 0.1064 - accuracy: 0.9705
Epoch 5/5
266/266 [==============================] - 14s 52ms/step - loss: 0.0535 - accuracy: 0.9858
{'loss': [2.514052152633667, 0.7974798083305359, 0.26762470602989197, 0.10644431412220001, 0.05353095754981041], 'accuracy': [0.22618907690048218, 0.7542054057121277, 0.9191467761993408, 0.9705132842063904, 0.985785961151123]}


In [144]:
model.evaluate(test_x, test_y)

47/47 [==============================] - 1s 14ms/step - loss: 0.7417 - accuracy: 0.8267


[0.741707980632782, 0.8266666531562805]